In [126]:
import os, csv, time, urllib, collections, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplot inline

ERROR:root:Line magic function `%matplot` not found.


In [127]:
raw = pd.read_csv('Building_Permits___Current.csv')
raw.head()

,Application/Permit Number,Permit Type,Address,Description,Category,Action Type,Work Type,Value,Applicant Name,Application Date,Issue Date,Final Date,Expiration Date,Status,Contractor,Permit and Complaint Status URL,Master Use Permit,Latitude,Longitude,Location
0,6578806,Construction,706 N 77TH ST,Construct interior alterations on main floor o...,SINGLE FAMILY / DUPLEX,ADD/ALT,No plan review,$50000.00,"DEHERRERA, LUCAS",01/30/2017,01/30/2017,NaN,07/30/2018,Permit Issued,NaN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,NaN,47.684927,-122.349437,"706 N 77TH ST\n(47.68492709, -122.34943726)"
1,6409682,Site Development,388R LAKESIDE AVE S,Shoreline Exemption to allow soil boring in st...,NaN,SHORELINE EXEMPTION ONLY,Plan Review,$0.00,"ANTIEAU, CLAY",01/30/2017,NaN,01/30/2017,NaN,AP Closed,CITY OF SEA SPU DRAIN & WASTE,http://web6.seattle.gov/dpd/PermitStatus/Proje...,NaN,47.599168,-122.286521,"388R LAKESIDE AVE S\n(47.59916842, -122.28652139)"
2,6612768,Site Development,10035 51ST AVE SW,Hazard tree removal of Alder causing allergic ...,NaN,TREE/VEGETATION MAINT/RESTORE,No plan review,$0.00,"SIVARAJAN, MURALI",09/18/2017,NaN,NaN,NaN,AP Closed,NaN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,NaN,47.512987,-122.394699,"10035 51ST AVE SW\n(47.51298692, -122.39469909)"
3,6529715,Construction,9820 TRITON DR NW,Removal of non-permitted roof-top (3rd floor) ...,SINGLE FAMILY / DUPLEX,ADD/ALT,Plan Review,$42000.00,"PETRISOR, CORNELL",06/13/2016,05/19/2017,06/28/2017,11/19/2018,Permit Closed,NaN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,NaN,47.700541,-122.391283,"9820 TRITON DR NW\n(47.70054059, -122.39128301)"
4,6437770,Construction,1015 36TH AVE E,Establish use as and construct a single family...,SINGLE FAMILY / DUPLEX,NEW,Plan Review,$709000.00,"GABBERT, JOHN",01/14/2015,05/27/2015,09/05/2017,12/31/2017,Permit Closed,NaN,http://web6.seattle.gov/dpd/PermitStatus/Proje...,NaN,47.627906,-122.287176,"1015 36TH AVE E\n(47.62790566, -122.28717586)"


In [ ]:
apiid = 'YOURAPIID'
apicode = 'YOURAPICODE'

poi = raw[['Application/Permit Number', 'Latitude', 'Longitude']]
poi.head()

## create output csv
path_to_poi = os.path.join('POI.csv')
with open(path_to_poi, 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Application/Permit Number', \
                     'num_poi_50m', 'num_poi_100m',\
                    'num_poi_150m', 'num_poi_500m'])

for i in range(len(raw)):
    permit = raw.iloc[i]['Application/Permit Number']
    lat = raw.iloc[i]['Latitude']
    lon = raw.iloc[i]['Longitude']

    ## 1. Nearby POIs (50m)
    baseurl = 'https://places.api.here.com/'
    path = 'places/v1/'
    resource = 'discover/here?'
    para = 'at={}%2C{}&app_id={}&app_code={}'.format(lat, lon, apiid, apicode)
    link = ''.join([baseurl, path, resource, para])
    #print link
    response = urllib.urlopen(link).read()
    data = json.loads(response)
    C_50 = len(data['results']['items'])
    #print C, 'POIs'       

    ## 2 more POIs
    ## number of POIs beyond 50m buffer, 100m, 150m, 500m
    baseurl = 'https://places.api.here.com/'
    path = 'places/v1/'
    resource = 'discover/explore?'
    category = ['eat-drink', 'going-out', 'sights-museums',
                'transport', 'accommodation', 'shopping',
               'leisure-outdoor', 'administrative-areas-buildings', 'natural-geographical',
               'atm-bank-exchange', 'hospital-health-care-facility']

    para = 'at={}%2C{}&cat={}&size=100&app_id={}&app_code={}'.format(lat, lon, ','.join(category), apiid, apicode)
    link = ''.join([baseurl, path, resource, para])
    response = urllib.urlopen(link).read()
    data = json.loads(response)

    C = len(data['results']['items'])
    C_100 = 0; C_150 = 0; C_500 = 0
    for c in range(C):
        if data['results']['items'][c]['distance'] <= 500:
            C_500 += 1
            if data['results']['items'][c]['distance'] <= 150:
                C_150 += 1
                if data['results']['items'][c]['distance'] <= 100:
                    C_100 += 1

    # write poi count to csv
    with open(path_to_poi, 'a') as outfile:
        writer = csv.writer(outfile, delimiter=',')
        writer.writerow([permit, C_50, C_100, C_150, C_500])
        
    time.sleep(0.1)
        
    if i % 1000 == 0:
        print 'Processed {:.2f}%'.format(i*100.0/len(raw))

In [89]:
timedf = raw[['Application/Permit Number', 'Application Date', \
            'Issue Date', 'Expiration Date']]

timedf['Application Date'] = pd.to_datetime(timedf['Application Date'])
timedf['Issue Date'] = pd.to_datetime(timedf['Issue Date'])
timedf['Expiration Date'] = pd.to_datetime(timedf['Expiration Date'])

timedf['To_Issue'] = timedf['Issue Date'] - timedf['Application Date']
timedf['Issue_Dur'] = timedf['Expiration Date'] - timedf['Issue Date']

def format_days(x):
    if ' days' in str(x):
        return int(x.days)
    else:
        return np.nan

timedf['To_Issue'] = timedf.To_Issue.apply(lambda x: format_days(x))
timedf['Issue_Dur'] = timedf.Issue_Dur.apply(lambda x: format_days(x))

timedf['Month_To_Issue'] = np.ceil(timedf['To_Issue'] / 30.0)
timedf['Month_To_Expire'] = np.ceil(timedf['Issue_Dur'] / 30.0)

#timedf.head()

timedf['Month'] = pd.DatetimeIndex(timedf['Application Date']).month

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=timedf['Application Date'].min(), \
                        end=timedf['Application Date'].max())
timedf['Holiday'] = timedf['Application Date'].isin(holidays)
timedf['Holiday'].value_counts()

timedf['Holiday_Season'] = 0
timedf.loc[timedf.Month >= 11, 'Holiday_Season'] = 1
timedf['House_Season'] = 0
timedf.loc[(timedf.Month >= 5) & (timedf.Month < 9), 'House_Season'] = 1

#timedf['Holiday_Season'].value_counts()
#timedf['House_Season'].value_counts()

time_dummy = pd.get_dummies(timedf['Month'], dummy_na=True)
timedf = pd.concat([timedf, time_dummy], axis=1)

timedf['Year'] = pd.DatetimeIndex(timedf['Application Date']).year
timedf['Year_diff'] = timedf['Year'] - 2000

#timedf.to_csv('temporal_feature.csv', index=False)

In [145]:
# join POI file
#from os import listdir
from os.path import isfile, join
poi_files = [f for f in os.listdir('.') if 'POI' in f]

In [149]:
poi_all = None

for f in poi_files:
    df = pd.read_csv(f)
    if poi_all is None:
        poi_all = df.copy()
    else:
        poi_all = pd.concat([poi_all, df], axis=0)

In [151]:
poi_all.head()

,Application/Permit Number,num_poi_50m,num_poi_100m,num_poi_150m,num_poi_500m
0,6578806.0,1.0,0.0,0.0,2.0
1,6409682.0,1.0,0.0,0.0,0.0
2,6612768.0,0.0,0.0,0.0,0.0
3,6529715.0,0.0,0.0,0.0,0.0
4,6437770.0,1.0,0.0,0.0,0.0


In [153]:
#poi_all.to_csv('poi_dense_feature.csv', index=False)

In [ ]:
zipcode = pd.read_csv('zipcode_feature.csv')

In [165]:
zipcode.head()

,Applicatio,ZIPCODE,98101.0,98102.0,98103.0,98104.0,98105.0,98106.0,98107.0,98108.0,...,98144.0,98146.0,98154.0,98155.0,98164.0,98177.0,98178.0,98195.0,98199.0,nan
0,6578806,98103,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6409682,98144,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,6612768,98146,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6529715,98117,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6437770,98112,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
zipcode = zipcode[['Applicatio', 'ZIPCODE']]

In [164]:
zipcode_dummy = pd.get_dummies(zipcode['ZIPCODE'], dummy_na=True)
zipcode = pd.concat([zipcode, zipcode_dummy], axis=1)

In [166]:
zipcode.shape

(56324, 34)

In [170]:
zipcode.to_csv('zipcode_feature.csv')#, index=False)

In [168]:
zipcode.head()

,Applicatio,ZIPCODE,98101.0,98102.0,98103.0,98104.0,98105.0,98106.0,98107.0,98108.0,...,98144.0,98146.0,98154.0,98155.0,98164.0,98177.0,98178.0,98195.0,98199.0,nan
0,6578806,98103,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6409682,98144,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,6612768,98146,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6529715,98117,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6437770,98112,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [187]:
allfeature = pd.merge(raw[['Application/Permit Number', 'Category']], zipcode, how='left',\
                left_on='Application/Permit Number', \
                right_on='Applicatio')

In [188]:
del allfeature['Category']

In [205]:
allfeature.head()

,Application/Permit Number,Applicatio,ZIPCODE,98101.0,98102.0,98103.0,98104.0,98105.0,98106.0,98107.0,...,98144.0,98146.0,98154.0,98155.0,98164.0,98177.0,98178.0,98195.0,98199.0,nan
0,6578806,6578806.0,98103.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6409682,6409682.0,98144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6612768,6612768.0,98146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6529715,6529715.0,98117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6437770,6437770.0,98112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
del allfeature['Applicatio']
del allfeature['ZIPCODE']

,Application/Permit Number,Applicatio,ZIPCODE,98101.0,98102.0,98103.0,98104.0,98105.0,98106.0,98107.0,...,98144.0,98146.0,98154.0,98155.0,98164.0,98177.0,98178.0,98195.0,98199.0,nan
5288,6619221,6619221.0,98104.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5289,6619221,6619221.0,98104.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5294,6619221,6619221.0,98104.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5295,6619221,6619221.0,98104.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
#allfeature.to_csv('zipcode_feature.csv')

In [222]:
#allfeature.head().to_csv('zipcode_head.csv', index=False)

In [233]:
#timedf.iloc[:,10:].head().to_csv('temporal_head.csv', index=False)

In [234]:
other = raw[['Application/Permit Number', 'Category', 'Permit Type', 'Action Type']]

In [236]:
other.to_csv('other_feature.csv', index=False)